In [28]:
import requests
import base64
import pandas as pd
import IPython.display as ipd

In [29]:
url_base = 'https://api.spotify.com'
urls_ep = { 'token' : 'https://accounts.spotify.com/api/token',
            'search' : url_base + '/v1/search',
            'artists' : {
               'artist' : url_base + '/v1/artists/{artist_id}', # Get an Artist > artist
               'top_tracks' : url_base + '/v1/artists/{artist_id}/top-tracks', # Get an Artist's Top Tracks > tracks
               'related_artist' : url_base + '/v1/artists/{artist_id}/related-artists', # Get an Artist's Related Artists > artists
               'albums' : url_base + '/v1/artists/{artist_id}/albums' # Get an Artist's Albums > albums
                },
            'albums' : {
                'album' : url_base + '/v1/albums/{album_id}', # Get an Album > album
                'tracks' : url_base + '/v1/albums/{album_id}/tracks' # Get an Album's Tracks > tracks
                },
            'playlists' : {
                'playlist' : url_base + '/v1/playlists/{playlist_id}', # Get a Playlist > playlist
                'current_user' : url_base + '/v1/me/playlists', # Get a List of Current User's Playlists > playlists
                'user' : url_base + '/v1/users/{user_id}/playlists', # Get a List of a User's Playlists > playlists
                'tracks' : url_base + '/v1/playlists/{playlist_id}/tracks' # Get a Playlist's Tracks > tracks
                },
            'tracks' : {
                'analyst' : url_base + '/v1/audio-analysis/{track_id}', # Get Audio Analysis for a Track > audio analysis object
                'features' : url_base + '/v1/audio-features/{track_id}', # Get Audio Features for a Track > audio features
                'track' : url_base + '/v1/tracks/{track_id}' # Get a Track > track
            }
           }

In [30]:
credentials = {
    'id' : 'e2db9b6cc74c4497a880d3cf4a4b0ea6',
    'secret' : '12d72ea59ad6492daf31ec163792a979',
    }
credentials['encode'] = base64.b64encode( bytes( credentials['id'] + ':' + credentials['secret'], 'utf8' ) )

token_data = {
    'params' : {
        'grant_type' : 'client_credentials'
        },
    'header' : {
        'Authorization' : 'Basic ' + str(credentials['encode'], 'utf8')
    }
}

search_types = ['artist', 'album', 'playlist', 'track', 'show', 'episode']
params_default = {'market' : 'MX'}

In [31]:
def _validate_connection(req):
    if req.status_code == 400:
        print(f'Invalid URL:\t {req.url}')
        return 400
    
    if req.status_code == 401:
        global token_auth
        token_auth = _get_token()
        print(f'Refreshing token')
        return 401

    if req.status_code == 200:
        print('Connection looks good!!')
        return 200

    print(f'The req to url {req.url} has thrown an error')
    print(req.json())
    return None

        

In [32]:
def _get_token():
    req = requests.post(urls_ep['token'], headers=token_data['header'], data=token_data['params'])
    print(f'token status:\t{req.status_code}')
    
    if _validate_connection(req) == 401:
        print('Unable to obtain token authorization')
        return 401

    print(f'Token valid for:\t {req.json()["expires_in"]} seg.')
    return {'Authorization' : req.json()['token_type'] + ' ' + req.json()['access_token']}


In [33]:
def search(search_text, type_results='artist'):
    search_params = {'q' : search_text, 
                    'type' : type_results, 
                    'market' : 'MX',
                    'limit' : 50}

    find_result = requests.get(urls_ep['search'], headers=token_auth, params=search_params)
    
    if _validate_connection(find_result) == 401:
        search(search_text, type_results)
    
    return find_result.json()[search_types[index_of_search_types] + 's']['items']

In [34]:
token_auth = _get_token()

token status:	200
Connection looks good!!
Token valid for:	 3600 seg.


In [35]:
search_types

['artist', 'album', 'playlist', 'track', 'show', 'episode']

In [86]:
search_text = 'Adele'
index_of_search_types = 0

find_result = search(search_text, search_types[index_of_search_types])


Connection looks good!!


In [87]:
artist_df = pd.DataFrame(find_result)
artist_df.drop(['external_urls', 'followers', 'href', 'images', 'uri'], axis='columns', inplace=True)
artist_df.head(2)

,genres,id,name,popularity,type
0,"[british soul, pop, uk pop]",4dpARuHxo51G3z768sgnrY,Adele,84,artist
1,"[norwegian pop, scandipop]",2qS6cYzM5ajhprcxQa1Ilc,Adelén,41,artist


In [88]:
def _is_it_the_type(evaluated_object, expected_type):
    try:
        if evaluated_object['type'] != expected_type: 
            print(f'The object type is not a/an {expected_type} is of type: {evaluated_object["type"]}')
            return False
        else:
            return True

    except KeyError:
        print(f'The object does not have a "type" property')
        return None

In [89]:
def get_albums_by_artist(artist):
    valid_type = _is_it_the_type(artist, 'artist')
    if valid_type is None or not valid_type: 
        return None

    albums_url = urls_ep['artists']['albums'].format(artist_id=artist['id'])
    albums_req = requests.get(albums_url, headers=token_auth, params=params_default)

    if _validate_connection(albums_req) == 401:
        get_albums_by_artist(artist)

    return albums_req.json()['items']

In [90]:
artist = find_result[0]
albums = get_albums_by_artist(artist)

Connection looks good!!


In [91]:
albums_df = pd.DataFrame(albums)
albums_df.drop(['album_group', 'album_type', 'artists', 'external_urls', 'href', 'images', 'release_date_precision', 'uri'], axis=1, inplace=True)

In [92]:
albums_df.head(2)

,id,name,release_date,total_tracks,type
0,0K4pIOOsfJ9lK8OjrZfXzd,25,2016-06-24,11,album
1,1azUkThwd2HfUDdeNeT147,21,2011-01-19,11,album


In [93]:
def get_tracks_by_album(album):
    valid_type = _is_it_the_type(album, 'album')
    if valid_type is None or not valid_type:
        return None

    tracks_url = urls_ep['albums']['tracks'].format(album_id=album['id'])
    tracks_req = requests.get(tracks_url, headers=token_auth, params=params_default)

    if _validate_connection(tracks_req) == 401:
        get_albums_by_artist(album)

    return tracks_req.json()['items']

In [94]:
album = albums[0]
tracks = get_tracks_by_album(album)


Connection looks good!!


In [95]:
tracks_df = pd.DataFrame(tracks)
tracks_df.drop(['artists', 'explicit', 'external_urls', 'href', 'is_local', 'is_playable', 'uri'], axis='columns', inplace=True)

In [96]:
tracks_df.head(2)

,disc_number,duration_ms,id,name,preview_url,track_number,type
0,1,295493,4sPmO7WMQUAf45kwMOtONw,Hello,https://p.scdn.co/mp3-preview/0b90429fd554bad6...,1,track
1,1,223080,4BHzQ9C00ceJxfG16AlNWb,Send My Love (To Your New Lover),https://p.scdn.co/mp3-preview/2f7c4f6de9f5a25e...,2,track


In [97]:
if tracks[0]['preview_url'] is not None:
    cancion_preview = requests.get(tracks[0]['preview_url'])
    ipd.Audio(cancion_preview.content)
else: 
    print('No hay url para preview de musica')